In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


In [ ]:
url = r"C:\Users\MSI\Desktop\DS\ayedi\fromage.xlsx"
data = pd.read_excel(url)
data


In [ ]:
print(data.head()) 

In [ ]:
print(data.describe()) 

In [ ]:
data.isnull().sum()
print(data)

In [ ]:
import seaborn as sns, matplotlib.pyplot as plt

numeric_data = data.select_dtypes(include=['int64', 'float64'])

corr_matrix = numeric_data.corr() 

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Heatmap de la matrice de corrélation')
plt.show()


In [ ]:
# Sélection des caractéristiques pour la classification
# Il s'agit de sélectionner les colonnes à utiliser comme caractéristiques dans le processus de classification.
# Dans ce cas, toutes les colonnes sauf la première sont sélectionnées, la première colonne étant probablement le nom du fromage.
features = data.iloc[:, 1:]

# Standardisation des données
# La standardisation des données est une étape importante dans de nombreux algorithmes d'apprentissage automatique,
# car elle garantit que les caractéristiques ont la même échelle, ce qui peut améliorer les performances du modèle.
scaler = StandardScaler()
# La méthode fit_transform calcule la moyenne et l'écart-type de chaque colonne (caractéristique) et effectue une transformation
# pour mettre à l'échelle les données afin qu'elles aient une moyenne de 0 et un écart-type de 1.
features_scaled = scaler.fit_transform(features)


In [ ]:
# Classification ascendante hiérarchique (CAH)
linkage_matrix = linkage(features_scaled, method='ward', metric='euclidean')

# Affichage du dendrogramme
plt.figure(figsize=(10, 6))
dendrogram(linkage_matrix)
plt.title('Dendrogramme CAH')
plt.xlabel('Fromages')
plt.ylabel('Distance')
plt.show()


In [ ]:
from sklearn.decomposition import PCA
# Importer la classe PCA (Analyse en Composantes Principales) depuis scikit-learn

# Appliquer PCA (Analyse en Composantes Principales)
pca = PCA(n_components=2)  # Définir le nombre de composantes principales à 2
features_pca = pca.fit_transform(features_scaled)
# La méthode fit_transform calcule les composantes principales à partir des données standardisées.

# Classification avec K-means
kmeans = KMeans(n_clusters=4, random_state=42)  # Choisir le nombre de clusters
kmeans.fit(features_scaled)
cluster_labels = kmeans.labels_
# Utilisation de l'algorithme K-means pour regrouper les données standardisées en 4 clusters.
# Les étiquettes des clusters sont stockées dans cluster_labels.

# Affichage du nuage de points avec PCA et coloration par classe
plt.figure(figsize=(10, 6))
for i in range(len(set(cluster_labels))):  # Parcourir chaque classe unique
    plt.scatter(features_pca[cluster_labels == i, 0],  # Sélectionner les données appartenant à la classe i
                features_pca[cluster_labels == i, 1],
                label=f'Cluster {i + 1}',  # Étiquette de la légende
                alpha=0.7)  # Opacité des points
plt.title('Nuage de points avec PCA (K-means clustering)')
plt.xlabel('Composante principale 1')
plt.ylabel('Composante principale 2')
plt.legend()
plt.show()
# Tracer un nuage de points en utilisant les deux premières composantes principales (axes) obtenues par PCA.
# Chaque point est coloré en fonction de son cluster assigné par l'algorithme K-means.
# Les légendes indiquent les clusters correspondants.


In [ ]:
from sklearn.decomposition import PCA
# Importer la classe PCA (Analyse en Composantes Principales) depuis scikit-learn

# Méthode du coude pour trouver le nombre optimal de clusters
inertia = []
for k in range(1, 10):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(features_scaled)
    inertia.append(kmeans.inertia_)
# La méthode du coude est une technique pour déterminer le nombre optimal de clusters dans un algorithme de clustering.
# Elle consiste à calculer l'inertie (somme des carrés des distances des échantillons au centre de leur cluster le plus proche)
# pour différentes valeurs de k (nombre de clusters). Ici, nous itérons sur différentes valeurs de k de 1 à 9.

# Tracer le graphique de la méthode du coude
plt.figure(figsize=(10, 6))
plt.plot(range(1, 10), inertia, marker='o', linestyle='-')
plt.title('Méthode du coude')
plt.xlabel('Nombre de clusters')
plt.ylabel('Inertie')
plt.xticks(range(1, 10))
plt.grid(True)
plt.show()
# Tracer un graphique montrant le changement de l'inertie en fonction du nombre de clusters.
# Le coude dans le graphique représente le nombre optimal de clusters, où l'inertie commence à diminuer de manière marginale.

# Choisir le nombre optimal de clusters à partir du graphique de la méthode du coude
optimal_k = int(input("Entrez le nombre optimal de clusters à partir du graphique de la méthode du coude : "))

# Classification avec K-means en utilisant le nombre optimal de clusters
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
kmeans.fit(features_scaled)
data['cluster_kmeans'] = kmeans.labels_
# Utiliser l'algorithme K-means avec le nombre optimal de clusters pour regrouper les données.
# Les étiquettes des clusters sont ajoutées à la colonne 'cluster_kmeans' du DataFrame.

# Affichage des centres des clusters
print("Centres des clusters (K-means):")
print(pd.DataFrame(scaler.inverse_transform(kmeans.cluster_centers_), columns=features.columns))
# Afficher les centres de chaque cluster en inversant la standardisation des centres des clusters,
# puis afficher ces centres avec les noms des caractéristiques originales.

